# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [152]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [153]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")


# Display sample data
city_data_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [154]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    size='Humidity', 
    tiles='OSM', 
    color='Humidity',
    frame_width=800,
    frame_height=600,
    hover_cols=['City', 'Country', 'Humidity']
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [155]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] < 28) & 
    (city_data_df['Max Temp'] > 20) &
    (city_data_df['Humidity'] <= 75) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample 

ideal_weather_df.head(20)



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
32,32,cape town,-33.9258,18.4232,20.61,71,0,9.26,ZA,1666108250
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
73,73,nalut,30.3333,10.8500,25.41,27,0,5.11,LY,1666108275
146,146,puerto del rosario,28.5004,-13.8627,24.86,73,0,4.63,ES,1666108035
152,152,kahului,20.8947,-156.4700,23.80,60,0,3.09,US,1666108246
240,240,tikaitnagar,26.9500,81.5833,23.56,59,0,0.35,IN,1666108378
258,258,comodoro rivadavia,-45.8667,-67.5000,25.92,24,0,7.20,AR,1666108389
265,265,san quintin,30.4833,-115.9500,21.20,74,0,1.37,MX,1666108394
340,340,santa rosalia,27.3167,-112.2833,24.62,56,0,0.74,MX,1666108436


### Step 3: Create a new DataFrame called `hotel_df`.

In [156]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head(20)


,City,Country,Lat,Lng,Humidity,Hotel Name
9,saint-pierre,RE,-21.3393,55.4781,68,
32,cape town,ZA,-33.9258,18.4232,71,
63,banda,IN,25.4833,80.3333,52,
73,nalut,LY,30.3333,10.8500,27,
146,puerto del rosario,ES,28.5004,-13.8627,73,
152,kahului,US,20.8947,-156.4700,60,
240,tikaitnagar,IN,26.9500,81.5833,59,
258,comodoro rivadavia,AR,-45.8667,-67.5000,24,
265,san quintin,MX,30.4833,-115.9500,74,
340,santa rosalia,MX,27.3167,-112.2833,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [157]:
# Set parameters to search for a hotel
radius = 10000
limit= 30
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
   # humidity = ... 
    #hotel_df.loc[index, 'humidity'] = humidity

    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
cape town - nearest hotel: Townhouse Hotel
banda - nearest hotel: #acnindiafy21
nalut - nearest hotel: No hotel found
puerto del rosario - nearest hotel: Hotel Tamasite
kahului - nearest hotel: Maui Seaside Hotel
tikaitnagar - nearest hotel: No hotel found
comodoro rivadavia - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Sol y Mar
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
lakki marwat - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso
pali - nearest hotel: Hotel Pawan International


,City,Country,Lat,Lng,Humidity,Hotel Name
9,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel
32,cape town,ZA,-33.9258,18.4232,71,Townhouse Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
73,nalut,LY,30.3333,10.8500,27,No hotel found
146,puerto del rosario,ES,28.5004,-13.8627,73,Hotel Tamasite
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
258,comodoro rivadavia,AR,-45.8667,-67.5000,24,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Sol y Mar


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [158]:
%%capture --no-display
# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    size='Humidity', 
    tiles='OSM', 
    color='Humidity',
    frame_width=800,
    frame_height=600,
    hover_cols=['city', 'country', 'humidity', 'Hotel Name']
)

# Display the map
hotel_map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name)